In [ ]:
# https://gis.stackexchange.com/questions/294206/create-a-polygon-from-coordinates-in-geopandas-with-python
# https://gist.github.com/rohanpsingh/0aa3d35d8e038a9c2491c68894f0b93b

from skimage import measure
from PIL import Image
import numpy as np
import geopandas as gpd
import glob
from shapely.geometry import Polygon
Image.MAX_IMAGE_PIXELS = None

directoryProcessed = './FullInferenceGrahamProcessed/'

for file in glob.glob("./FullInferenceGrahamProcessed/*_vessel.png"):
    
    (filename, x) = os.path.splitext(os.path.basename(file))
    name = filename.split("_")[0]
    print(name)
    

    im = Image.open(file)
    imarray = np.array(im)

    crs = {'init': 'epsg:4326'}
    first = True


    sub_mask = imarray == 1
    contours = measure.find_contours(sub_mask, 0.5, positive_orientation='low')


    polygons = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        #poly = poly.simplify(1.0, preserve_topology=True)
        polygons.append(poly)

    print(len(polygons))

    polygon = gpd.GeoDataFrame(index=[], crs=crs, geometry=[])  
    for p in polygons:
        polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[p])     
        polygon = polygon.geometry.append(polygon.geometry)    

    polygon.to_file(filename=directoryProcessed + name + '_vessel.geojson', driver='GeoJSON')
